# Installing libraries

In [3]:
!pip install transformers accelerate peft bitsandbytes "qwen-vl-utils[decord]==0.0.8"
# or use the 2U1 repo's environment.yaml/requirements.txt for comprehensive setup

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 135.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 135.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 170.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.2/39.2 MB 160.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 155.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12/12 [peft]2m11/12 [peft]erate]s]ub]


# Fine-Tuning

In [ ]:
# torchrun --nproc_per_node=2 run_finetune.py \
#   --model qwen/Qwen2.5-VL-7B-Instruct \
#   --data_dir path/to/your/data \
#   --output_dir output/qwen-vet \
#   --lora_rank 8 \
#   --vision_tune adapter \
#   --...other params...
